In [ ]:
!pip install layer_sdk --upgrade

     |████████████████████████████████| 471 kB 5.3 MB/s 
     |████████████████████████████████| 1.3 MB 52.8 MB/s 
     |████████████████████████████████| 2.4 MB 26.2 MB/s 
     |████████████████████████████████| 4.4 MB 30.8 MB/s 
     |████████████████████████████████| 381 kB 44.9 MB/s 
     |████████████████████████████████| 159 kB 43.5 MB/s 
     |████████████████████████████████| 56 kB 3.3 MB/s 
     |████████████████████████████████| 256 kB 35.8 MB/s 
     |████████████████████████████████| 16.8 MB 1.7 MB/s 
     |████████████████████████████████| 40 kB 5.3 MB/s 
     |████████████████████████████████| 97 kB 7.0 MB/s 
     |████████████████████████████████| 26.7 MB 15.8 MB/s 
     |████████████████████████████████| 4.0 MB 31.7 MB/s 
     |████████████████████████████████| 212 kB 45.3 MB/s 
     |████████████████████████████████| 132 kB 49.4 MB/s 
     |████████████████████████████████| 271 kB 34.4 MB/s 
     |████████████████████████████████| 4.0 MB 10.9 MB/s 
     |██████████████

In [23]:
!rm -rf ecommerce_order_review_score_prediction
!git clone https://github.com/layerml/ecommerce_order_review_score_prediction

Cloning into 'ecommerce_order_review_score_prediction'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.


In [28]:
#from layer.v2.assertions import greatexpectations, assert_true, assert_valid_values, assert_not_null, assert_unique
from layer.decorators import dataset, model,resources, pip_requirements, assertions, fabric
from layer.decorators.assertions import assert_unique, assert_valid_values
from layer.client import Dataset, Model
import layer

#layer.logout()
layer.login()
#layer.login("https://development.layer.co/")

# ***ML PROJECT: Predict review score of an order based on its information***

# **Data Transformation**

In [35]:
import pandas as pd
import numpy as np

@resources("ecommerce_order_review_score_prediction/olist_orders_dataset.csv")
@dataset("orders_raw_table")
def load_order_table():
  orders_df = pd.read_csv("ecommerce_order_review_score_prediction/olist_orders_dataset.csv")

  layer.log({"Dataset Description": "Raw orders table by loading from a csv file"})

  return orders_df

@dataset("orders_clean_table",dependencies=[Dataset('orders_raw_table')])
def clean_order_table():
  # Load dataset
  orders_df = layer.get_dataset("orders_raw_table").to_pandas()

  # Drop all the rows having at least 1 null value - Since there are just a few null values in the data, we could just drop all of them.
  df = orders_df.dropna()

  # We will do our analysis only on the delivered orders
  df = df[df['order_status'] == 'delivered']

  # Drop rows that don't meet the requirement: order_purchase_timestamp <= order_approved_at <= order_delivered_carrier_date <= order_delivered_customer_date
  df = df[~((df['order_purchase_timestamp'] >= df['order_approved_at']) | (df['order_approved_at'] >= df['order_delivered_carrier_date']) | (df['order_delivered_carrier_date'] >= df['order_delivered_customer_date']))]

  layer.log({"Dataset Description": "Cleaned version of the orders table by dropping na rows, selecting only 'delivered' orders and doing sanity checks on timestamp columns."})

  return df


def bucketize_actual_delivery_vs_expectation (row):
  if row['days_between_delivery_expectation'] <= 0 :
    return -1
  elif row['days_between_delivery_expectation'] <= 7 :
    return 1
  elif row['days_between_delivery_expectation'] <= 14 :
    return 2
  else:
    return 3

@dataset("orders_based_features",dependencies=[Dataset('orders_clean_table')])
def extract_features_order_table():
  # Load dataset
  df = layer.get_dataset("orders_clean_table").to_pandas()

  # Days between purchase and delivery dates
  df['days_between_purhcase_and_delivery'] = (pd.to_datetime(df['order_delivered_customer_date']) - pd.to_datetime(df['order_purchase_timestamp'])).dt.days

  # if the order was approved late or on time (0=on time, 1=late)
  df['order_approved_late']=np.where((pd.to_datetime(df['order_approved_at']) - pd.to_datetime(df['order_purchase_timestamp'])).dt.days == 0, 0, 1)

  # Actual delivery vs. Expected delivery: 1=Delivered before expected date, 2= Delivered one week later than expected date, 3= Delivered two weeks later than expected date, 4= Delivered more than two weeks later than expected date
  df['days_between_delivery_expectation']=(pd.to_datetime(df['order_estimated_delivery_date']) - pd.to_datetime(df['order_delivered_customer_date'])).dt.days
  df['actual_delivery_vs_expectation_bucket'] = df.apply (lambda row: bucketize_actual_delivery_vs_expectation(row), axis=1)

  layer.log({"Dataset Description": "Features extracted only from the orders table"})
  layer.log({"days_between_purchase_and_delivery":"Days between delivery date and purchase date",
             "order_approved_late":"0: Order payment is approved on the same day with purchase -- 1: Otherwise",
             "actual_delivery_vs_expectation_bucket":"It represents the days between estimated delivery date and actual delivery date -- 1: Less than 7 days, 2: Less than 14 days more than 7 days, 3: More than 14 days, -1: Order delivered later than estimated date",
             "order_delivered_carrier_date":"The date order delivered to carrier"
             })

  df = df[['order_id','days_between_purhcase_and_delivery','order_approved_late','actual_delivery_vs_expectation_bucket','order_delivered_carrier_date']]
  return df

@resources("ecommerce_order_review_score_prediction/olist_order_items_dataset.csv")
@dataset("items_raw_table")
def load_item_table():
  # Load items table from csv file
  items_df = pd.read_csv("ecommerce_order_review_score_prediction/olist_order_items_dataset.csv")

  layer.log({"Dataset Description": "Raw items table by loading from a csv file"})

  return items_df  


@dataset("items_clean_table",dependencies=[Dataset('items_raw_table')])
def clean_items_table():
  # Load dataset
  items_df = layer.get_dataset("items_raw_table").to_pandas()

  # Select relevant columns and drop any na valued rows
  df = items_df[['order_id','shipping_limit_date','price','freight_value']].dropna()

  # Price and Freight Value must be non-negative
  df = df[(items_df['price']>=0) & (items_df['freight_value']>=0)]

  layer.log({"Dataset Description": "Cleaned version of the raw items table by selecting some relevant columns out of it, dropping na rows and doing some sanity checks on 'price' and 'freight_value' columns"})

  return df

@dataset("items_based_features",dependencies=[Dataset('items_clean_table')])
def extract_features_items_table():
  # Load dataset
  df = layer.get_dataset("items_clean_table").to_pandas()
  # Extract 3 features: total_order_price & total_order_freight
  df1 = df.groupby('order_id').agg(total_order_price=('price', 'sum'), total_order_freight=('freight_value','sum'),max_shipping_limit_date=('shipping_limit_date','max')).reset_index()

  # Extract 1 feature: is_multiItems_order -- If the order has multiple items or not (1 or 0)
  df2 = df.groupby('order_id').agg(cnt=('price', 'count')).reset_index()
  df2['is_multiItems_order'] = np.where(df2['cnt'] > 1, 1, 0)

  df3 = df1.merge(df2, how="inner", on='order_id')[['order_id','is_multiItems_order','total_order_price','total_order_freight','max_shipping_limit_date']]

  layer.log({"Dataset Description": "Features extracted only from the items table"})
  layer.log({"total_order_price":"Total price paid for the order",
             "total_order_freight":"Total price paid for freight transport",
             "max_shipping_limit_date":"Maximum of expected shipping date if order has many items",
             "is_multiItems_order":"If the order has multiple items or not. 0: Single item order -- 1: Multiple items order"
             })

  return df3

@resources("ecommerce_order_review_score_prediction/olist_order_reviews_dataset.csv")
@dataset("reviews_raw_table")
def load_reviews_table():
  # Load the reviews table from csv file
  reviews_df = pd.read_csv("ecommerce_order_review_score_prediction/olist_order_reviews_dataset.csv")

  layer.log({"Dataset Description": "Raw reviews table by loading from a csv file"})

  return reviews_df

@dataset("reviews_clean_table",dependencies=[Dataset('reviews_raw_table')])
def clean_reviews_table():
  # Load dataset
  reviews_df = layer.get_dataset("reviews_raw_table").to_pandas()

  # Drop extra reviews if an order has multiple order review scores
  df = reviews_df.groupby('order_id', as_index= False).agg(review_score=('review_score', 'max'))

  layer.log({"Dataset Description": "This table is used to create labels (target variable) which is review scores."})
  
  return df

@assert_unique(["order_id"])
@assert_valid_values("review_score", [1,2,3,4,5])
@dataset("training_data",dependencies=[Dataset('orders_based_features'),Dataset('items_based_features'),Dataset('reviews_clean_table')])
def generate_training_data():
  from functools import reduce
  # Merge all clean datasets
  orders_data = layer.get_dataset("orders_based_features").to_pandas()
  items_data = layer.get_dataset("items_based_features").to_pandas()
  reviews_data = layer.get_dataset("reviews_clean_table").to_pandas()

  data_frames = [orders_data, items_data, reviews_data]
  df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['order_id'],how='inner'), data_frames)

  # Create a new feature: seller_shipped_late
  df_merged['days_between_delivered_carrier_and_shipping_limit'] = (pd.to_datetime(df_merged['order_delivered_carrier_date']) - pd.to_datetime(df_merged['max_shipping_limit_date'])).dt.days
  df_merged['seller_shipped_late'] = np.where(df_merged['days_between_delivered_carrier_and_shipping_limit'] > 0 , 1 ,0) 

  # Select only relevant columns (features)
  df_merged = df_merged.drop(columns=['order_delivered_carrier_date', 'max_shipping_limit_date','days_between_delivered_carrier_and_shipping_limit'])

  layer.log({"Dataset Description": "All features from the orders and items tables. Final training dataset."})
  layer.log({"order_id":"Unique id for the order",
             "days_between_purchase_and_delivery":"Days between delivery date and purchase date",
             "order_approved_late":"0: Order payment is approved on the same day with purchase -- 1: Otherwise",
             "actual_delivery_vs_expectation_bucket":"Days between estimated delivery date and actual delivery date -- 1: Less than 7 days, 2: Less than 14 days more than 7 days, 3: More than 14 days, -1: Order delivered later than estimated date",
             "total_order_price":"Total price paid for the order",
             "total_order_freight":"Total price paid for freight transport",
             "is_multiItems_order":"If the order has multiple items or not. 0: Single item order -- 1: Multiple items order",
             "seller_shipped_late":"if seller shipped items later than promised date. 1: Late - 0: Before or on time",
             "review_score":"Review score for the order between 1 and 5",
             })
  
  
  return df_merged


# **Model Functions Refined**

In [45]:
import xgboost
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import time

def data_split(data):
  data = layer.get_dataset("training_data").to_pandas()

  training_data, testing_data = train_test_split(data, test_size=0.1, random_state=42)

  # split data into train and test sets
  X_train, X_valid, y_train, y_valid = train_test_split(training_data.drop(['review_score', 'order_id'], axis=1), training_data['review_score'], test_size=0.1, random_state=7)

  data_pair = [(X_train, y_train), (X_valid, y_valid)]

  return data_pair


def model_overfit_check(X_train, y_train, eval_set):
  param_dict = {
    'colsample_bytree' : 1.0,
    'learning_rate': 0.02,
    'max_depth': 5,
    'min_child_weight': 10,
    'subsample' : 0.5
  }

  xgb_model = XGBRegressor(n_estimators=1000, objective='reg:squarederror', colsample_bytree = param_dict['colsample_bytree'], learning_rate=param_dict['learning_rate'], max_depth=param_dict['max_depth'], min_child_weight=param_dict['min_child_weight'], subsample=param_dict['subsample'])

  xgb_model.fit(X_train, y_train, eval_metric='rmse', eval_set=eval_set, verbose=False)

  # retrieve performance metrics
  results = xgb_model.evals_result()
  epochs = len(results['validation_0']['rmse'])
  x_axis = range(0, epochs)

  # plot rmse - train vs. test
  fig, ax = plt.subplots()
  ax.plot(x_axis, results['validation_0']['rmse'], label='Train')
  ax.plot(x_axis, results['validation_1']['rmse'], label='Test')
  ax.legend()
  ax.set_ylim([1.135, 1.18])
  plt.ylabel('Root Mean Square Error')
  plt.title('XGBoost RMSE')
  fig = plt.gcf()

  # Layer logs the plot
  layer.log({"Train vs Test - Model Overfit Check": fig})

  # clear all plots and figures from memory
  plt.figure().clear()
  plt.close()
  plt.cla()
  plt.clf()


def check_model_performance(xgb_model: XGBRegressor, test_data_X, test_data_Y):
  # PLOT 1: make predictions and show in a bar distribution plot
  yhat = xgb_model.predict(test_data_X)
  plt.hist(yhat)
  fig1 = plt.gcf()

  # Layer logs the plot
  layer.log({"Test Data Predicted Review Score Distribution": fig1})

  # clear all plots and figures from memory
  plt.figure().clear()
  plt.close()
  plt.cla()
  plt.clf()

  # PLOT 2: distribution of actual review scores
  plt.hist(test_data_Y)
  fig2 = plt.gcf()

  # Layer logs the plot
  layer.log({"Test Data Real Review Score Distribution": fig2})

  # clear all plots and figures from memory
  plt.figure().clear()
  plt.close()
  plt.cla()
  plt.clf()

@pip_requirements(packages=["xgboost==0.90"])
@fabric("f-medium")
@model("review_score_predictor_model",dependencies=[Dataset('training_data')])
def train_final_model():
  # The best parameter combination
  param_dict = {
    'colsample_bytree' : 1.0,
    'learning_rate': 0.02,
    'max_depth': 5,
    'min_child_weight': 10,
    'subsample' : 0.5
  }

  # Layer logs model description and model parameters
  layer.log({"Model Description" : "XGBRegressor with squared error objective function to predict review scores of orders based on their high level features."})
  layer.log(param_dict)

  training_data = layer.get_dataset("training_data").to_pandas()
  data_pair = data_split(training_data)

  train_data_X = data_pair[0][0]
  train_data_Y = data_pair[0][1]

  test_data_X = data_pair[1][0]
  test_data_Y = data_pair[1][1]

  xgb_model_final = XGBRegressor(objective='reg:squarederror', n_estimators=200, colsample_bytree = param_dict['colsample_bytree'], learning_rate=param_dict['learning_rate'], max_depth=param_dict['max_depth'], min_child_weight=param_dict['min_child_weight'], subsample=param_dict['subsample'])
  xgb_model_final.fit(train_data_X, train_data_Y,verbose=False)

  model_overfit_check(train_data_X,train_data_Y, data_pair)
  check_model_performance(xgb_model_final, test_data_X, test_data_Y)


  return xgb_model_final


In [39]:
## LAYER Project Initialize
layer.init('ecommerce_olist_order_review_score_prediction',
           fabric="f-small",
           pip_packages=["numpy","sklearn","pandas"]
           )

## LAYER REMOTE MODE
layer.run([load_order_table,
           clean_order_table,
           extract_features_order_table,
           load_item_table,
           clean_items_table,
           extract_features_items_table,
           load_reviews_table,
           clean_reviews_table,
           generate_training_data,
           train_final_model], debug=True)

## LAYER LOCAL MODEL - Run your functions in order
# load_order_table()
# clean_order_table()
# extract_features_order_table()
# load_item_table()
# clean_items_table()
# extract_features_items_table()
# load_reviews_table()
# clean_reviews_table()
# generate_training_data()
# train_final_model()

Output()

14:08:29 items_raw_table: Starting setup of dependencies...
14:08:29 items_raw_table: Using selector: EpollSelector
14:08:29 items_raw_table: Successfully logged into https://app.layer.ai
14:08:29 items_raw_table: Using selector: EpollSelector
14:08:29 items_raw_table: Downloading execution artifacts data-catalog--layer2022040715233801730000001e/add1b570-c8e7-4187-b747-1d01104893a9/ecommerce_olist_order_review_score_prediction/items_raw_table/9f1c7c8c-8366-4c91-b13c-cea6b00fb38a/items_raw_table.tgz to ~/source
14:08:29 items_raw_table: Creating directory ~/source
14:08:29 items_raw_table: Place __init__.py in ~/source
14:08:29 items_raw_table: Download binary(add1b570-c8e7-4187-b747-1d01104893a9/ecommerce_olist_order_review_score_prediction/items_raw_table/9f1c7c8c-8366-4c91-b13c-cea6b00fb38a/items_raw_table.tgz) to temp directory
14:08:30 items_raw_table: Binary archive items_raw_table.tgz downloaded and extracted to ~/source successfully
14:08:30 items_raw_table: Execution artifacts 

Run(project_name='ecommerce_olist_order_review_score_prediction')

In [41]:
import layer

my_model = layer.get_model('layer/ecommerce_olist_order_review_score_prediction/models/review_score_predictor_model:2.1').get_train()

df = layer.get_dataset('layer/ecommerce_olist_order_review_score_prediction/datasets/training_data:1.2').to_pandas()


In [44]:
test_sample = df.drop(['review_score', 'order_id'], axis=1).sample()
predicted_review_score = layer.get_model("review_score_predictor_model").get_train().predict(test_sample)
print("PREDICTED REVIEW SCORE [1-5]: ",predicted_review_score)

PREDICTED REVIEW SCORE [1-5]:
 [1.6939433]

ORDER FEATURES: 
        days_between_purhcase_and_delivery  order_approved_late  \
65905                                  57                    0   

       actual_delivery_vs_expectation_bucket  is_multiItems_order  \
65905                                     -1                    0   

       total_order_price  total_order_freight  seller_shipped_late  
65905              49.95                16.79                    0  
